In [1]:
import os
import sys
sys.path.append(os.getcwd())

In [2]:
from compare import StanzaNERFacade
from compare import SpacyNERFacade

/Users/pgasiorowski/Documents/Projects/axent/ner-compare/projectenv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
text = ''
with open('data/fida.md') as f:
    text = f.read().rstrip()

In [4]:
nlps = {
    'Stanza-Default': StanzaNERFacade(),
    'Spacy-Default': SpacyNERFacade()
}

/Users/pgasiorowski/Documents/Projects/axent/ner-compare/projectenv/lib/python3.12/site-packages/stanza/models/tokenization/trainer.py:82: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental

In [5]:


for nlp_name, nlp in nlps.items():
    ents = nlp.transform(text)
    print(f'## {nlp_name}')
    for ent in ents:
        print('* {} ({})'.format(ent.get('text'), ent.get('type')))

## Stanza-Default
* 28 grudnia 2023 r. (date)
* 27 grudnia 2023 r. (date)
* Spółka (orgName)
* DFG (orgName)
* „WFS" (orgName)
* Niemiec (placeName)
* Niemczech (placeName)
* „Grupa" (orgName)
* Rzeczypospolitej Polskiej (placeName)
* Polsce (placeName)
* Spółki (orgName)
* Spółce (orgName)
* Grupy DFG (orgName)
* Niemczech (placeName)
* DFG (orgName)
* DFG (orgName)
* DFG (orgName)
* DFG (orgName)
* DFG (orgName)
* WFS (orgName)
* WFS (orgName)
* DFG (orgName)
* WFS (orgName)
* DFG (orgName)
* DFG (orgName)
* WFS (orgName)
* WFS (orgName)
* DFG (orgName)
* DFG (orgName)
* WFS (orgName)
* DFG (orgName)
* WFS (orgName)
* DFG (orgName)
* WFS (orgName)
* DFG (orgName)
* DFG (orgName)
* WFS (orgName)
* DFG (orgName)
* WFS (orgName)
* DFG (orgName)
* WFS (orgName)
* DFG (orgName)
* DFG (orgName)
* DFG (orgName)
* WFS (orgName)
* WFS (orgName)
* DFG (orgName)
* Spółki (orgName)
* WFS (orgName)
* DFG (orgName)
* WFS (orgName)
* DFG (orgName)
* DFG (orgName)
* WFS (orgName)
* WFS (orgName)
* W